In [1]:
import gymnasium as gym
import ale_py
import numpy as np
#blackjack = gym.make
gym.register_envs(ale_py)
#for key in gym.envs.registry.keys():
#  print(key)

In [8]:
blackjack_env = gym.make("Blackjack-v1", render_mode="human")
observation, info = blackjack_env.reset()

episode_over = False
while not episode_over:
    # randomly sample an action
    action = blackjack_env.action_space.sample()

    # obtain partially observed state, reward, if the state is done, if we've exceeded time, and other info
    # by stepping through the environment using our action
    observation, reward, terminated, truncated, info = blackjack_env.step(action)
    print(episode_over)

    # if our state is terminal, or we have reached the time limit, end
    episode_over = terminated or truncated

# close our environment.
blackjack_env.close()

False
False


The above cell contains code to render a blackjack environment, and take random actions on it. It doesn't lead to very good performance. Let's change that using a simple RL algorithm.

Let's analyze the environment characteristics and see what we can do.

In [ ]:
blackjack_q_learning_env = gym.make("Blackjack-v1")


print(blackjack_q_learning_env.observation_space[0])
print(blackjack_q_learning_env.observation_space[1])
print(blackjack_q_learning_env.observation_space[2])
print(blackjack_q_learning_env.action_space)
flattened_number_of_states = 32 * 11 * 2
number_of_actions = 2
print(f"Flattened number of states: {flattened_number_of_states}")
print(f"Flattened number of actions: {number_of_actions}")

# initialize Q function
# we changed the way the Q function is initialized so it makes it easier 
# for us to access states
Q_function = np.zeros((32,11,2,number_of_actions))

# set number of epochs
num_epochs = 1000
# set number of timesteps per epoch
num_timesteps_per_epoch = 22


Discrete(32)
Discrete(11)
Discrete(2)
Discrete(2)
Flattened number of states: 704
Flattened number of actions: 2


In [ ]:
## main Q-function training loop on the blackjack environment

# NOTE: We'll need to find a way to save the trained policy
E = num_epochs
T = num_timesteps_per_epoch
for epoch in range(E):
  timestep = 0
  obs, info = blackjack_q_learning_env.reset()
  # get every state we visited in this epoch:
  list_of_states = []
  while timestep < T:
    list_of_states.append(obs)
    print(f"observation: {obs}")
    random_action = blackjack_q_learning_env.action_space.sample()
    print(f"random action: {random_action}")
    next_obs, reward, terminated, truncated, info = blackjack_q_learning_env.step(random_action)
    print(f"reward: {reward}")
    if terminated or truncated:
      break
    # do some work here to process next_obs
    obs = next_obs
    timestep += 1 # update the timestep to match the changed environment
  # we set the index to be the index before the last element of the list, because we handle
  # the case involving the last element of the list before we get into the while loop.
  reverse_state_list_index = len(list_of_states) - 2
  sc0,sc1,sc2 = list_of_states[-1]
  # update the Q function for the terminal state
  Q_function[sc0,sc1,sc2,random_action] += reward 
  while reverse_state_list_index > -1:
    # update the Q function for all the other, non-terminal states
    pass
  break
  

observation: (12, 3, 0)
random action: 1
reward: 0.0
observation: (19, 3, 0)
random action: 1
reward: -1.0
